In [21]:
import pandas as pd
import numpy as np
from Layer import *

df = pd.read_csv("./data/salary/train_sep.csv")
df_test = pd.read_csv("./data/salary/test_sep.csv")
test_x = df_test.values.astype('float').T #(dim,n)
#remember feature scaling
test_x = (test_x - np.mean(test_x,axis=1).reshape((-1,1)))/(np.std(test_x,axis=1)+ 1e-10).reshape((-1,1))


In [22]:
df_label = df.loc[:, 'income']
df_train = df.loc[:, :].drop('income', axis=1)

data_x = df_train.values.astype(np.float).T
data_y = df_label.values.astype(np.float).reshape((1, -1))

data_x = (data_x - np.mean(data_x, axis=1).reshape((-1, 1))) / np.std(data_x, axis=1).reshape((-1, 1))

In [23]:
def loss(pred, y):
    pred_log = np.log(pred + 1e-20)  # q(1) = f(x)
    no_pred_log = np.log(1.0 - pred + 1e-20)  # q(0) = 1 - f(x) ; f(x) : probability of class A(1)
    loss_val = -1. * (np.matmul(y, pred_log.T) + np.matmul((1.0 - y), no_pred_log.T))
    return loss_val

def accuarcy(pred,y):
    pred = np.rint(pred)
    return np.mean(np.equal(pred,y).astype('float'))

def grad_cross_entropy(pred,y):
    d_c_pred = -1.0 * (y / (pred + 1e-20) - (1. - y) / (1. - pred + 1e-20))  # (1,n) it is grad not loss
    return d_c_pred

def shuffle():
    seed = np.random.randint(0,1e5)
    
    np.random.seed(seed)
    #I make a big error : shuffle
    #np shuffle is shuffled by the first dim.in this example,is row
    #but we make the data_x [dim,n].so, we need transpose the data_x first and shuffle next
    np.random.shuffle(data_x.T)
    np.random.seed(seed)
    np.random.shuffle(data_y.T)

dim = data_x.shape[0]
dim1 = 64
dim2 = 16
dim3 = 4
dim4 = 2
dim5 = 1

layers = [Layer(dim,dim1,relu,relu_rev),
          Layer(dim1,dim3,relu,relu_rev),
          #Layer(dim2,dim3,relu,relu_rev),
          #Layer(dim3,dim4,relu,relu_rev),
          Layer(dim3,dim5,sigmoid,sigmoid_rev)]
n = np.floor(data_x.shape[1] * 0.6).astype("int")
lr = 0.1
epochs = 2000
batch_size = 256
batch_count = n // batch_size
loss_list = []

def predict(epoch):
    pred = test_x
    for layer in layers:
        pred = layer.forward(pred)
    
    pred = np.rint(pred).reshape((-1,))#<=50K : 0; >50K : 1
    pred_list = list(pred)
    id_list = range(1,len(pred)+1)
    df_out = pd.DataFrame({'id':id_list,'label':pred_list},dtype='int')
    df_out.to_csv("./result/salary/pred_{}.csv".format(epoch),index=False)

for epoch in range(epochs):
    loss_val = 0
    shuffle()
    
    for i in range(batch_count):
        a = data_x[:,i*batch_size:(i+1)*batch_size]
        y = data_y[:,i*batch_size:(i+1)*batch_size]
    
        # forward
        for layer in layers:
            a = layer.forward(a)
    
        # cross entropy
        d_c_a = grad_cross_entropy(a,y)  # (1,n) it is grad not loss
    
        # backpropagation
        for layer in reversed(layers):
            d_c_a = layer.backpropagation(d_c_a,lr)
        
        loss_val += loss(a, y)[0][0]
    
    loss_list.append(loss_val)
    a = data_x[:,n:]
    y = data_y[:,n:]
    for layer in layers:
            a = layer.forward(a)
    acc = accuarcy(a,y)
    if epoch %1 == 0:
            print("{:<5d},loss:{:.5f},acc={:.5f}".format(epoch,loss_val,acc))
    if (epoch+1) % 10 == 0:
        predict(epoch)
        print("save")

0    ,loss:7277.73500,acc=0.84468
1    ,loss:6520.49855,acc=0.84875
2    ,loss:6348.03117,acc=0.85528
3    ,loss:6276.55044,acc=0.86165
4    ,loss:6115.43732,acc=0.85850
5    ,loss:6080.35525,acc=0.86104
6    ,loss:5953.60597,acc=0.85935
7    ,loss:5911.66459,acc=0.86035
8    ,loss:5893.94511,acc=0.86173
9    ,loss:5783.09211,acc=0.86127
save
10   ,loss:5781.51622,acc=0.86173
11   ,loss:5742.28784,acc=0.86311
12   ,loss:5756.48239,acc=0.86879
13   ,loss:5641.22195,acc=0.86434
14   ,loss:5668.62359,acc=0.86779
15   ,loss:5632.05452,acc=0.86848
16   ,loss:5631.71582,acc=0.86756
17   ,loss:5617.73821,acc=0.86702
18   ,loss:5610.54802,acc=0.87086
19   ,loss:5539.29624,acc=0.86956
save
20   ,loss:5575.69948,acc=0.87086
21   ,loss:5461.13454,acc=0.86779
22   ,loss:5463.89735,acc=0.86917
23   ,loss:5545.76629,acc=0.87539
24   ,loss:5436.19728,acc=0.86956
25   ,loss:5510.44607,acc=0.87248
26   ,loss:5413.86711,acc=0.87094
27   ,loss:5444.78139,acc=0.87163
28   ,loss:5442.62345,acc=0.87547
29  

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.show()